# Current vs Density

In [1]:
import sys
import os
sys.path.insert(0, '../magcolloids')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import HTML, clear_output
import jsonpickle
import scipy.optimize as spo
import scipy.spatial as spa 
import magcolloids as mgc
import matplotlib as mpl
import string as st
import support as sp

from tqdm import tqdm_notebook

ureg = mgc.ureg

idx = pd.IndexSlice

%reload_ext autoreload
%autoreload 2

In [2]:
mpl.rc('text', usetex=True)
mpl.rcParams['figure.dpi'] = 150

In [3]:
directory = "Z:/Dimers/Simulations/DensityDependence3/"
index_name = "index.dat"

index = pd.read_csv(os.path.join(directory,index_name),sep=" ", names = ["file","packing"])
index.head()

,file,packing
0,Phi_0.01,0.010000
1,Phi_0.04379746835443038,0.043797
2,Phi_0.021265822784810127,0.021266
3,Phi_0.07759493670886075,0.077595
4,Phi_0.03253164556962025,0.032532


In [4]:
h = 3.9 * ureg.um
tilt_angle = 7*ureg.deg
frequency = 3*ureg.Hz

In [5]:
def differentiated_velocity(name):
    ## Lazy open the trajectory
    trj_read = mgc.trj_lazyread(os.path.join(directory,name.file+".lammpstrj"), 
                     output = ["x","y","z","mux","muy","muz","fx","fy"])

    ## Define a subset
    timestep = 1e-4*ureg.s
    frames = np.array(list(trj_read.T.keys()))
    times = frames*timestep
    ### We only read the last 15 seconds (out of 60), to ensure we are not in a transient
    readthis = times>45*ureg.s

    ## Load Trajectory
    trj = trj_read.read_trj(np.where(readthis))
    bounds = trj_read.get_bounds(slice(0,1,1))
    trj = trj.filter(["x","y","z"])
    trj["time"] = trj.index.get_level_values("frame")*timestep.to(ureg.s).magnitude
    trj["frequency"] = frequency

    ## Calculate Velocity Array
    velocities = sp.calculate_velocities(trj, bounds)

    ## Calculate Differentiated Velocity
    velocities["plane"] = np.NaN
    velocities.loc[trj.z>0,"plane"] = "up"
    velocities.loc[trj.z<0,"plane"] = "down"
    vupdown = velocities.groupby(["plane"]).mean().filter(["x","y"])
    vupdown.to_csv(os.path.join(directory,name.file+"_vupdown.dat"), sep="\t")
    return vupdown

In [6]:
diff_vel = [differentiated_velocity(name) for i,name in tqdm_notebook(index.loc[:].iterrows())]